<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_reportA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML reportA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




----
## 準備
----


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

---
### California Housing Dataset

この課題では，California housing dataset というデータセットを使う．
これは，アメリカカリフォルニアの住宅価格を予測するという回帰問題のデータセットである．

参考
- https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset
- https://atmarkit.itmedia.co.jp/ait/articles/2201/31/news042.html


#### データの入手

scikit-learn の [sklearn.datasets.fetch_california_housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) 関数を呼ぶと，このデータセットを入手できる．

In [ ]:
# as_frame を True にすると pandas.DataFrame 形式で取得できる
data = fetch_california_housing(as_frame=True)
dfX = data['data']
dfX

見て分かるように，8つの値から成るデータが20640件ある．これらは回帰モデルへの入力となる．
1件のデータは，一定の領域内に存在する住宅および住人について求めた次の8つの値から成る（詳細は↑の参考リンク先参照）．

1. MedInc: 所得
1. HouseAge: 築年数
1. AveRooms: 部屋数
1. AveBedrms: 寝室数
1. Population: 人口
1. AveOccup: 世帯数
1. Latitude: 緯度
1. Longitude: 経度

予測対象となる住宅価格は，次のような値である．

In [ ]:
dfY = data['target']
dfY

#### 学習データと検証データの分割


[sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) を使って学習データと検証データに分割する．




In [ ]:
# 全データの 8 割を学習データとし，残りを検証データとする
#   random_state を指定しなければ実行のたびに分割の仕方が変わるので毎回異なる実験結果となるが，
#   ここでは一つの値に固定しているで，何度やっても（当然，誰がやっても）同じ条件で実験できる
XL, XV, yL, yV = train_test_split(dfX.to_numpy(), dfY.to_numpy(), train_size=0.8, random_state=0)
print(XL.shape, yL.shape, XV.shape, yV.shape)

#### 前処理

入力となる8つの変数のばらつき方が変数ごとに違いすぎるので，各変数の平均が 0 で分散が 1 となるようにする前処理（標準化）を適用する．
ここでは，[sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) を使う．

In [ ]:
# XLの変数ごとの平均と分散を求める
scaler = StandardScaler()
scaler.fit(XL)
print(scaler.mean_) # 平均
print(scaler.var_) # 分散

# 求めた平均と分散を使って XL, XV を標準化
XL2 = scaler.transform(XL)
XV2 = scaler.transform(XV)

---
## 線形回帰
---


★ 下のコードセルに，次のことをするコードを書きなさい

1. 標準化した学習データを使って線形回帰モデルを学習させる． scikit-learn の LinearRegression を使おう
1. 学習データと検証データのそれぞれについて，住宅価格の予測値を求め，予測値と正解の値との間の平均二乗誤差を算出する





---
## ニューラルネットを用いた回帰
---

今度は，ニューラルネットを使ってみよう．ニューラルネットとはどんなものかについては，この授業の後の回で解説予定なので，現時点では深い理解は求めない．
以下の実験で分かるように，「ニューロンの数」がモデルのハイパーパラメータで，これを変えるとモデルの複雑度が変化することを知っていればok．

【参考】
-  2023年度「機械学習I」第5回 https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023#ex05
-  2023年度「機械学習I」第6回 https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023#ex06

---
### 2層ニューラルネットの実験

次のコードを実行すると，中間層が1つの階層型ニューラルネット（2層ニューラルネット）を学習させることができる．
中間層のニューロン数は変数 `H` で指定している．
ニューラルネットの学習では，目的関数を最適化するための処理を何度も繰り返す必要があり，その繰り返し回数（の最大値）をここでは 50 としている．



In [ ]:
H = 16
model = MLPRegressor(hidden_layer_sizes=(H,), activation='relu', alpha=0.0, max_iter=50, verbose=True)
regressor = model.fit(XL2, yL)

# この下にコードを追加しよう


```
ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
```
という警告が出るかもしれないが，学習の繰り返し回数が最大値に達したと言っているだけなので気にせずともよい．

★ 上のコードセルに，次のことを行うコードを追加しなさい．

> 学習データと検証データのそれぞれについて，上記で学習したニューラルネットによって得られる住宅価格の予測値を求め，予測値と正解の値との間の平均二乗誤差を算出して表示する

★ ニューラルネットでは，パラメータの初期値によって学習結果が変化する．上記のコードではパラメータがランダムに初期化されるので，実行のたびに異なる結果が得られる．そのことをふまえて，次のことをやりなさい．

1. コードセルを5回実行し，各回に得られた平均二乗誤差の値をこのセルに書き込む
1. それらを線形回帰の実験で得られた結果と比較して簡単に考察する（このセルに書き込めばよい）






★ 上記の2層ニューラルネットでは，中間層に含まれるニューロンの数を変えるとモデルの複雑度が変化する．上のコードで変数 `H` で指定しているニューロン数を 256 に変えて，↑のセルと同じことをしなさい（5回繰り返して結果を考察）．



ニューラルネットには多数のハイパーパラメータが存在するが，ここでは簡単のため，中間層のニューロン数のみを考慮している．

---
### 3層ニューラルネットの実験

中間層を2つに増やした，3層のニューラルネットで同じような実験をやってみよう．
2つの層のニューロン数は独立に変化させることができるが，それだと組み合わせが膨大になるので，ここでは，2つの中間層が同数のニューロンで構成されるもののみを考える．

また，ニューラルネットはパラメータの初期値によって学習結果が変わるので，同じ構造のニューラルネットでも本当は初期値を何通りか変えて実験するべきである（2層の場合には5回ずつ実行していた）．
しかし，複雑なニューラルネットの学習には時間がかかるので，ここでは1回のみでよしとする．

次のコードは，2つの中間層にそれぞれ `H` 個のニューロンをもつ3層ニューラルネットワークを定義している．
2層の場合よりも学習の繰返し回数を増やし（`max_iter=200`），学習途上の出力を抑制している（`verbose=False`）．
また，結果を再現できるよう．パラメータの初期化に使う乱数の種を固定している（`random_state=0`）．

In [ ]:
H = 16
model = MLPRegressor(hidden_layer_sizes=(H,H,), activation='relu', alpha=0.0, max_iter=200, verbose=False, random_state=0)

# この下にコードを追加しよう


★ 上のコードセルに，次のことを行うコードを追加しなさい．

> `XL2` と `yL` を用いて学習を行い，学習データと検証データのそれぞれについて，住宅価格の予測値を求め，予測値と正解の値との間の平均二乗誤差を算出して表示する

★ 上の実験の条件のうち，中間層のニューロン数のみを 32, 64, 128, 256, 512 に変えて実験を行いなさい．上のコードセルの `H` の値を手動で書きかえて実行を繰り返すかわりに，次のコードセルの中身を書こう．

In [ ]:
HList = [16, 32, 64, 128, 256, 512]

for H in HList:
    # モデルの定義
    # 学習
    # 学習データに対する平均二乗誤差の算出
    # 検証データに対する平均二乗誤差の算出
    # H と2つの平均二乗誤差を print


★ 上記の実験結果についての考察をこのテキストセルに書きなさい．

